In [43]:
import pandas as pd
df = pd.read_csv(r"C:\Users\aniru\Downloads\Lumaa_Assignment\mpst_dataset\mpst_full_data.csv")
df.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [50]:
train_df = df[df["split"] == "train"]
train_df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'split',
       'synopsis_source'],
      dtype='object')

In [27]:
def load_data(path):
    df = pd.read_csv(path)
    return df

In [87]:
def process_dataframe(df):

    reqd_df = df[["imdb_id", "title", "plot_synopsis", "tags"]]
    reqd_df["tags"] = reqd_df["tags"].apply(lambda x: x.replace(",", " "))
    reqd_df["textual_information"] = reqd_df["tags"] + " " + reqd_df["plot_synopsis"]

    return reqd_df


In [50]:
reqd_df = process_dataframe(df)
print(reqd_df.shape)

(14828, 5)


C:\Users\aniru\AppData\Local\Temp\ipykernel_32820\927830703.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reqd_df["tags"] = reqd_df["tags"].apply(lambda x: x.replace(",", " "))
C:\Users\aniru\AppData\Local\Temp\ipykernel_32820\927830703.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reqd_df["textual_information"] = reqd_df["tags"] + " " + reqd_df["plot_synopsis"]


In [53]:
reqd_df.head()

,imdb_id,title,plot_synopsis,tags,textual_information
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,cult horror gothic murder atmospheric,cult horror gothic murder atmospheric Note...
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,"violence Two thousand years ago, Nhagruul the ..."
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,"romantic Matuschek's, a gift store in Budapest..."
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...",inspiring romantic stupid feel-good,inspiring romantic stupid feel-good Glenn H...
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...",cruelty murder dramatic cult violence atm...,cruelty murder dramatic cult violence atm...


In [54]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")

nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [89]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import contractions

# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()

from nltk.stem import PorterStemmer 
# Create a Porter Stemmer instance
porter_stemmer = PorterStemmer()
 
def preprocess_tokens(sent):

    stop_words = dict.fromkeys(list(set(stopwords.words('english'))), 1)


    word_tokens = word_tokenize(sent)
    filtered_sentence = [contractions.fix(w.lower()) for w in word_tokens if stop_words.get(w) is None and w.isnumeric() is False]
    filtered_sentence_lemma = [porter_stemmer.stem(w) for w in filtered_sentence]
    
    return " ".join(filtered_sentence_lemma)




In [16]:
reqd_df["processed_textual_information"] = reqd_df["textual_information"].apply(lambda x: preprocess_tokens(x))


In [10]:
reqd_df.shape

(14828, 6)

In [17]:
reqd_df.to_csv("../processed_data/processed_data.csv", index=False)

In [61]:
final_df = pd.read_csv(r"../processed_data/processed_data_random.csv")

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize a tfidf object
tfidf = TfidfVectorizer(max_features=3000)

# Transform the data
vectorized_data = tfidf.fit_transform(final_df['processed_textual_information'].values)


dictionary = dict(zip(tfidf.get_feature_names_out(), list(tfidf.idf_)))



In [64]:
import pickle
pickle.dump(tfidf, open(r"../pickle_files/vectorizer.pkl", "wb"))

pickle.dump(vectorized_data, open(r"../pickle_files/vectorized_data.pkl", "wb"))

In [2]:
import pandas as pd
df = pd.read_csv(r"../mpst_dataset/mpst_full_data.csv")

In [3]:
df

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb
...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",test,wikipedia
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",train,wikipedia
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,train,wikipedia
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,test,wikipedia


In [10]:
global dk
dk = {}

def tags(ind, str_):

    for tag in str_.split(","):
        if dk.get(tag.strip()) is None:
            dk[tag.strip()] = [ind]
        else:
            dk[tag.strip()] += [ind]
    return


for index, str_ in enumerate(df["tags"].values.tolist()):
    tags(index, str_)


dk = {k: v for k, v in sorted(dk.items(), key=lambda item: len(item[1]), reverse=True)}

for k,v in dk.items():
    print(k, len(v))


murder 5782
violence 4426
flashback 2937
romantic 2906
cult 2647
revenge 2468
psychedelic 1897
comedy 1859
suspenseful 1086
good versus evil 875
humor 822
satire 816
entertaining 749
neo noir 745
action 664
sadist 652
insanity 636
tragedy 597
fantasy 564
paranormal 546
boring 525
mystery 519
horror 486
melodrama 457
cruelty 442
gothic 441
dramatic 412
dark 405
atmospheric 396
storytelling 364
sci-fi 309
psychological 289
historical 272
absurd 270
prank 255
sentimental 233
philosophical 228
avant garde 220
bleak 211
depressing 205
plot twist 205
alternate reality 205
realism 204
cute 197
stupid 190
intrigue 168
pornographic 163
home movie 153
haunting 151
historical fiction 143
allegory 139
adult comedy 128
thought-provoking 120
inspiring 118
anti war 118
comic 114
brainwashing 107
alternate history 102
queer 98
clever 87
claustrophobic 84
whimsical 79
feel-good 76
blaxploitation 74
western 73
grindhouse film 66
suicidal 54
magical realism 54
autobiographical 44
christian film 42
non fi

In [96]:
import pickle
pickle.dump(dk, open(r"../pickle_files/genrewise_data.pkl", "wb"))

In [79]:
import random
def random_sample(reqd_df, dk, top_k=10, size= 100):

    final_dataset_indices = []

    visited_set = set()

    for index, (k,v) in enumerate(dk.items()):
                
        if index > 9:
            break
        
        flag = 0
        while not flag:

            prev_ = len(visited_set)

            indices = random.sample(v, size)

            visited_set.update(set(indices))

            new_  = len(visited_set)

            diff = new_ - prev_

            if (diff - len(indices)) <= 2:
                flag = 1
                final_dataset_indices.extend(indices)

    final_df = reqd_df.iloc[final_dataset_indices]

    return final_df
         


In [82]:
df_final = random_sample(reqd_df, dk, top_k=10, size= 100)

In [83]:
df_final.shape

(1000, 5)

In [85]:
df_final

,imdb_id,title,plot_synopsis,tags,textual_information
10250,tt0043117,Wagon Master,The film opens with a prelude showing a murder...,murder,murder The film opens with a prelude showing a...
1681,tt0257044,Road to Perdition,"In 1931, during the Great Depression, Michael ...",boring neo noir depressing murder violence...,boring neo noir depressing murder violence...
7838,tt0067243,Assault,"In January 1945, as the Second World War in Eu...",murder,"murder In January 1945, as the Second World Wa..."
14446,tt0058401,Night Must Fall,Mrs. Bramson (Dame May Whitty) is an irascible...,murder,murder Mrs. Bramson (Dame May Whitty) is an ir...
13658,tt0113010,Fair Game,Valerie Plame is employed by the Central Intel...,violence cult comedy murder sadist,violence cult comedy murder sadist Valerie...
...,...,...,...,...,...
4215,tt2140315,Raaz 3: The Third Dimension,The Movie is on Jealousy and what an Human do ...,good versus evil revenge,good versus evil revenge The Movie is on Jeal...
13598,tt0435687,The League of Gentlemen's Apocalypse,"One night, Jeremy Dyson calls the other member...",good versus evil comedy humor murder,good versus evil comedy humor murder One ni...
13558,tt0096071,The Serpent and the Rainbow,"In 1978, a Haitian man named Christophe (Conra...",good versus evil cult psychedelic,"good versus evil cult psychedelic In 1978, a..."
161,tt0317919,Mission: Impossible III,Ethan Hunt has retired from field work for the...,murder violence good versus evil psychedeli...,murder violence good versus evil psychedeli...


In [90]:
df_final["processed_textual_information"] = df_final["textual_information"].apply(lambda x : preprocess_tokens(x))

C:\Users\aniru\AppData\Local\Temp\ipykernel_32820\712875287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["processed_textual_information"] = df_final["textual_information"].apply(lambda x : preprocess_tokens(x))


In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize a tfidf object
tfidf = TfidfVectorizer(max_features=1000, min_df=5)

# Transform the data
vectorized_data = tfidf.fit_transform(df_final['processed_textual_information'].values)


dictionary = dict(zip(tfidf.get_feature_names_out(), list(tfidf.idf_)))

dictionary

{'000': 3.235925944853314,
 'abandon': 2.8653296623959736,
 'abl': 2.483804761833818,
 'accept': 2.651259407287439,
 'accid': 3.274025791085585,
 'accident': 2.833580964081394,
 'accus': 3.274025791085585,
 'across': 2.528857425774761,
 'act': 2.5664205273504095,
 'action': 2.6357552207514736,
 'activ': 3.1296312862036912,
 'actual': 2.51968304949872,
 'adam': 4.0169344812045935,
 'admit': 2.9183221925364844,
 'advanc': 3.18136696060288,
 'affair': 3.3654599970452166,
 'aft': 3.334043800811838,
 'after': 1.5303285956636339,
 'afterward': 3.1380701548495558,
 'agent': 2.981501094158016,
 'ago': 3.1296312862036912,
 'agre': 2.1819070317280236,
 'air': 3.1465808445174646,
 'airport': 3.781620394270129,
 'alfr': 5.13616605707544,
 'alic': 4.541458949328746,
 'alien': 3.9967317738870745,
 'aliv': 2.6564813512685905,
 'all': 3.1129642337184795,
 'allow': 2.379325691803797,
 'almost': 2.9599948889370524,
 'alon': 2.640896620251892,
 'along': 2.2457942991792748,
 'alreadi': 2.8653296623959736,

In [29]:
def compute_top_5_titles(sim_, df):

    """Get the top 5 recommendations and their scores."""
    score_dict = {}
    sorted_indices = np.argsort([x[0] for x in sim_])[::-1]
    top_5 = sorted_indices[:5]

    for ind in top_5:
        score_dict[df.iloc[ind]["title"]] = (sim_[ind][0], df.iloc[ind]["tags"])
    
    return score_dict

In [92]:
def preprocess_tokens(lemmatizer, sent):
    """Perform Lemmatizations and contractions on the detaset. Remove stopwords etc.."""
    stop_words = dict.fromkeys(list(set(stopwords.words('english'))), 1)


    word_tokens = word_tokenize(sent)
    filtered_sentence = [contractions.fix(w.lower()) for w in word_tokens if stop_words.get(w) is None and w.isnumeric() is False]
    filtered_sentence_lemma = [lemmatizer.stem(w) for w in filtered_sentence]
    
    return " ".join(filtered_sentence_lemma)

In [93]:
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity

sent = "I love thrilling action movies set in space, with a comedic twist"

lemmatizer = WordNetLemmatizer()

# preprocess user-query
preprocessed_sent = preprocess_tokens(porter_stemmer, sent)
# vectorize user-query
vector = tfidf.transform([preprocessed_sent])
# compute cosine similarity
sim_= cosine_similarity(vectorized_data, vector)
# return top 5 similar movies
score_dict = compute_top_5_titles(sim_, reqd_df)

In [62]:
type(vectorized_data)

scipy.sparse._csr.csr_matrix

In [94]:
print(score_dict)

{'Captain Kronos - Vampire Hunter': (0.25835130799103373, 'good versus evil  action  gothic'), 'A Most Violent Year': (0.2313928526657974, 'violence'), 'Your Friends & Neighbors': (0.22091628900271054, 'comedy  cruelty  satire'), 'Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan': (0.2107989777203483, 'comedy  boring  adult comedy  bleak  cult  flashback  humor  satire  romantic  entertaining'), 'Thralls': (0.20172809209901807, 'pornographic  violence  murder  flashback')}


In [34]:
df_final

,imdb_id,title,plot_synopsis,tags,textual_information,processed_textual_information
2485,tt0091419,Little Shop of Horrors,The narrator (Stanley Jones) tells of an event...,comedy dark murder cult romantic sadist,comedy dark murder cult romantic sadist T...,comedy dark murder cult romantic sadist the na...
11292,tt0406956,Macbeth,=== Act I ===\nThe play opens amidst thunder a...,tragedy avant garde murder,tragedy avant garde murder === Act I ===\nTh...,tragedy avant garde murder === act i === the p...
6553,tt0051673,Good Day for a Hanging,Eddie Campbell (Robert Vaughn) watches a stage...,murder,murder Eddie Campbell (Robert Vaughn) watches ...,murder eddie campbell ( robert vaughn ) watch ...
4687,tt0053337,Teenagers from Outer Space,"Derek (Derek Love), a teenaged alien guy, amon...",murder,"murder Derek (Derek Love), a teenaged alien gu...","murder derek ( derek love ) , teenaged alien g..."
4472,tt0840361,The Town,This synopsis is based on the extended edition...,boring neo noir murder dramatic violence ...,boring neo noir murder dramatic violence ...,boring neo noir murder dramatic violence reven...
...,...,...,...,...,...,...
14354,tt0074452,The Eagle Has Landed,The book makes use of the false document techn...,suspenseful murder violence good versus evi...,suspenseful murder violence good versus evi...,suspenseful murder violence good versus evil h...
4534,tt0377109,The Ring Two,A nervous teen male tries to make his date wat...,good versus evil,good versus evil A nervous teen male tries to ...,good versus evil a nervous teen male try make ...
3645,tt0486576,4: Rise of the Silver Surfer,The movie opens out in space as a strange alie...,good versus evil murder,good versus evil murder The movie opens out i...,good versus evil murder the movie open space s...
5342,tt2027128,Justice League: Doom,"The Justice League, with Cyborg's assistance, ...",good versus evil flashback,good versus evil flashback The Justice League...,good versus evil flashback the justice league ...
